In [1]:
import json
import numpy as np
import torch
from torch_geometric.data import Data
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from transformers import pipeline


In [2]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [3]:
with open('edinburgh-keywords_train.json', 'r') as f:
    train_data = json.load(f)

keywords = list(train_data['np2count'].keys())

keyword_set = set(keywords)

In [4]:
def extract_users(info):
    l_user, user2kw = [], []
    for ii in info:
        lus = info[ii]
        for u in lus:
            if u not in l_user:
                l_user.append(u)
                user2kw.append([])
            idx = l_user.index(u)
            user2kw[idx].append(ii)
    return l_user, user2kw

In [5]:
train_users, train_users2kw = extract_users(train_data['np2users'])


In [6]:
restaurant_set = set()
listres = []
for kw in train_data['np2rests'].keys():
    listres.extend(train_data['np2rests'][kw].keys())
restaurant_set = set(listres)

keyword_set = list(keyword_set)
restaurant_set = list(restaurant_set)
restaurants = len(listres)
num_keywords = len(keyword_set)
num_restaurants = len(restaurant_set)
a = np.zeros((num_keywords, num_restaurants))

In [7]:
for kw in train_data['np2rests'].keys():
    for res in train_data['np2rests'][kw].keys():
        idx_kw = keyword_set.index(kw)
        idx_res = restaurant_set.index(res)
        a[idx_kw][idx_res] = 1

In [8]:
keyword_embeddings = model.encode(list(keyword_set))

In [9]:
with open('edinburgh-keywords_test.json', 'r') as r:
    test_data = json.load(r)

user_keywords = list(test_data['np2reviews'].keys())
user_keywords_list = list(user_keywords)

In [10]:
test_users, test_users2kw = extract_users(test_data['np2users'])


In [11]:
test_keywords = [kw for sublist in test_users2kw for kw in sublist]
test_keyword_embeddings = model.encode(test_keywords)

similarity_scores = cosine_similarity(test_keyword_embeddings, keyword_embeddings)

In [12]:
filtered_keywords = []
for i, user_kw in enumerate(test_users2kw):
    updated_user_kw = []
    for kw in user_kw:
        if kw not in keyword_set:
            test_idx = test_keywords.index(kw)
            sim_scores = similarity_scores[test_idx]

            best_match_idx = np.argmax(sim_scores)
            best_match_keyword = keyword_set[best_match_idx]

            updated_user_kw.append(best_match_keyword)
        else:
            updated_user_kw.append(kw)

    filtered_keywords.append(updated_user_kw)

test_users2kw = filtered_keywords

In [13]:
results = []
for kw in test_users2kw:
    t = np.zeros((1, len(keyword_set)))
    keywords = kw[:10]
    for keys in keywords:
        if keys in keyword_set:
            idx_kw = keyword_set.index(keys)
            t[0][idx_kw] = 1
    R = np.dot(t, a)
    result = np.argsort(R[0])[::-1][:10]
    results.append(result)

In [1]:
# if __name__ == "__main__":
#         for i, result in enumerate(results):
#             restaurant_names = [restaurant_set[idx] for idx in result]
#             print(f"The result for user {i} is: {restaurant_names}")

SỬ DỤNG LLM ĐỂ RE-RANK

In [34]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
llm_pipeline = pipeline("text-generation", model="gpt2")

def re_rank_candidates(user_id, candidate_restaurants, user_keywords):
    prompt = (
        f"Người dùng có sở thích: {', '.join(user_keywords[:5])}. "  
        f"Với các candidate restaurants sau: {', '.join(candidate_restaurants[:5])}. "  
        "Hãy xếp hạng lại các restaurant theo mức độ phù hợp với sở thích của người dùng theo thứ tự giảm dần và chỉ in ra danh sách tên các restaurant, cách nhau bằng dấu phẩy."
    )
    
    generated = llm_pipeline(prompt, max_new_tokens=200, do_sample=False)[0]['generated_text']
    
    output = generated[len(prompt):].strip()
    
    re_ranked = [restaurant.strip() for restaurant in output.split(',')]
    
    valid_re_ranked = [r for r in re_ranked if r in candidate_restaurants]
    if len(valid_re_ranked) == 0:
        valid_re_ranked = candidate_restaurants
    return valid_re_ranked

final_results = []
for idx, (user, candidate_indices) in enumerate(zip(test_users, results)):
    candidate_restaurants = [restaurant_set[i] for i in candidate_indices]
    user_kw = test_users2kw[idx]
    re_ranked = re_rank_candidates(user, candidate_restaurants, user_kw)
    final_results.append(re_ranked)
    print(f"Re-ranked results for user {user}: {re_ranked}")

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

c:\Users\Nhat Minh\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nhat Minh\.cache\huggingface\hub\models--Qwen--QwQ-32B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/63.2k [00:00<?, ?B/s]

model-00001-of-00014.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
assert openai_api_key, "OPENAI_API_KEY not found. Please check your .env file."
def re_rank_candidates(user_id, candidate_restaurants, user_keywords):
    prompt = (
        f"Người dùng có sở thích: {', '.join(user_keywords[:5])}. "
        f"Với các candidate restaurants sau: {', '.join(candidate_restaurants[:5])}. "
        "Hãy xếp hạng lại các restaurant theo mức độ phù hợp với sở thích của người dùng theo thứ tự giảm dần và chỉ in ra danh sách tên các restaurant, cách nhau bằng dấu phẩy."
    )
    
    try:
        # Use the openai.ChatCompletion.create for GPT-4
        response = openai.ChatCompletion.create(
            model="gpt-4",  # Correct model name
            messages=[
                {"role": "system", "content": "You are an assistant helping with restaurant recommendations."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200,  # Limit response length
            temperature=0.7  # Control response randomness
        )

        # Extract the generated content
        generated = response['choices'][0]['message']['content'].strip()
        
        # Parse the output (list of restaurant names separated by commas)
        re_ranked = [restaurant.strip() for restaurant in generated.split(',')]
        
        # Ensure the re-ranked list contains only valid candidate restaurants
        valid_re_ranked = [r for r in re_ranked if r in candidate_restaurants]
        if len(valid_re_ranked) == 0:
            valid_re_ranked = candidate_restaurants
        
        return valid_re_ranked
    except Exception as e:
        print(f"Error with GPT-4 API request: {e}")
        return candidate_restaurants  # Fallback to original if an error occurs

# Example usage for all users
final_results = []
for idx, (user, candidate_indices) in enumerate(zip(test_users, results)):
    candidate_restaurants = [restaurant_set[i] for i in candidate_indices]
    user_kw = test_users2kw[idx]
    re_ranked = re_rank_candidates(user, candidate_restaurants, user_kw)
    final_results.append(re_ranked)
    print(f"Re-ranked results for user {user}: {re_ranked}")


Error with GPT-4 API request: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

Re-ranked results for user yfXqZkU5iXE07GSHzdsQBA: ['V0tRCjf95FTqVJvmxdeqbQ', 'E7s0QreUJ2H9r0Ls1yDeiQ', '8PmbCVEjWGPouD1Tst6FsQ', 'z-3ZDmVbHZjDrYZE42yKZA', 'QMySP-qHdwQNZY0gDba0ng', 'EKEPtNKJdZxRhYoFhuG8ew', 'xbCNlp2vWixy2W1WsfGv8g', '0HyuO5rOKeaa08TFzjIJIA', 'W-kOZ946xRWBJZvxXhJ7DA', '8-HSr0TFRWW8tHDVHd_MbQ']
Error with GPT-4 API request: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the

In [ ]:
def generate_results(test_users, results, test_users2kw, restaurant_set, re_ranked):
    output_data = {}

    for idx, (user, restaurant_indices) in enumerate(zip(test_users, results)):
        user_data = {}

        user_keywords = test_users2kw[idx]

        candidate_restaurants = [restaurant_set[i] for i in restaurant_indices]

        re_ranked_restaurants = re_ranked[idx]

        positions = [str(i) for i in restaurant_indices]  

        user_data["kw"] = user_keywords[:10]  
        user_data["candidate"] = re_ranked_restaurants[:10] 
        user_data["positions"] = positions[:10]  
        output_data[user] = user_data

    return output_data 


result_dict = generate_results(test_users, results, test_users2kw, restaurant_set, final_results)
print(result_dict)  



Results saved to: ./result/results.json


In [ ]:
def save_results_to_json(test_users, results, test_users2kw, restaurant_set, re_ranked, file_path='./result/results(afterLLM).json'):
    output_data = {}

    for idx, (user, restaurant_indices) in enumerate(zip(test_users, results)):
        user_data = {}

        user_keywords = test_users2kw[idx]

        candidate_restaurants = [restaurant_set[i] for i in restaurant_indices]

        re_ranked_restaurants = re_ranked[idx]

        positions = [str(i) for i in restaurant_indices]  # 1-based index

        user_data["kw"] = user_keywords[:10]  
        user_data["candidate"] = re_ranked_restaurants[:10] 
        user_data["positions"] = positions[:10]  
        output_data[user] = user_data

    with open(file_path, mode="w", encoding="utf-8") as json_file:
        json.dump(output_data, json_file, ensure_ascii=False, indent=4)

    print(f"Results saved to: {file_path}")

save_results_to_json(test_users, results, test_users2kw, restaurant_set, final_results)


Results saved to: ./result/results(afterLLM).json


In [ ]:
save_results_to_json(test_users, results, test_users2kw, restaurant_set, final_results)
